In [1]:
import os

In [3]:
%pwd

'd:\\End-to-End-DeepLearning-Projects\\Coccidiosis-Disease-Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\End-to-End-DeepLearning-Projects\\Coccidiosis-Disease-Classification'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [18]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
             root_dir = config.root_dir,
             source_URL= config.source_URL,
             local_data_file= config.local_data_file,
             unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


In [19]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Functions returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-08-19 16:06:12,208: INFO: common: yaml file loaded from: config\config.yml successfully]
[2023-08-19 16:06:12,210: INFO: common: yaml file loaded from: params.yaml successfully]
[2023-08-19 16:06:12,211: INFO: common: created directory at: artifacts]
[2023-08-19 16:06:12,212: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-19 16:06:21,099: INFO: 1464286924: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4d99c30c434736659085fe524adb902b36b43a1c4ec98ce3bbe7d82a6ed15536"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A800:1D4B:216EDE:289B8B:64E09B1A
Accept-Ranges: bytes
Date: Sat, 19 Aug 2023 10:36:11 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10549-MRS
X-